In [6]:
import pandas as pd
import random as rd
import seaborn as sns

from planning_livreurs import *

from IPython.display import HTML, display, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Widget pour générer l'emploi du temps optimisé

In [7]:
list_streets = ['Champs-Elysées', 'Convention', 'Sts']

# On crée le dictionnaire avec des VBox vide :
dic_widgets_streets = {key: widgets.VBox() for key in list_streets}



## ------------------------------------------------------------------------------------------------


nb_jours = 7

dic_sliders_colis = dict(zip(list_streets, [[
        widgets.IntSlider(value=0,
                          min=0,
                          max=10,
                          step=1,
                          description=f'Jour {i}') for i in range(nb_jours)
    ] for _ in range(3)]))

for key in dic_widgets_streets.keys():
    # On associera le 1er élément de VBox.children comme étant relatif aux vendeurs
    # et le 2nd élément sera lié aux colis.
    dic_widgets_streets[key].children = [widgets.HTML(value=f'<b>{key} :</b>')]
    dic_widgets_streets[key].children += tuple(dic_sliders_colis[key])

hbox_children = list(dic_widgets_streets.values())
hbox = widgets.HBox(hbox_children)


## ------------------------------------------------------------------------------------------------



slider_livreurs = widgets.IntSlider(
    value=8,
    min=0,
    max=50,
    step=1,
    description='#livreurs'
)

shifts_slider = widgets.SelectionRangeSlider(
    options=list(range(0,10)),
    index=(4, 8),
    description='#shifts'
)

hbox_general = widgets.VBox([slider_livreurs, shifts_slider])


## ------------------------------------------------------------------------------------------------


tab = widgets.Tab([hbox_general, hbox])
tab.set_title(0, 'Général')
tab.set_title(1, 'Par rue')

## ------------------------------------------------------------------------------------------------


def get_list_prenoms_courants(k):
    df_prenoms = pd.read_csv('data/prenoms.csv')
    list_prenoms = df_prenoms['Garçons'].tolist() + df_prenoms['Filles'].tolist()
    
    return rd.sample(list_prenoms, k=k)



update_button = widgets.Button(
    description='Mettre à jour',
)

container = widgets.VBox([tab ,update_button])

@update_button.on_click
def update(b=None):
    # In order not to have 2 tables if the user clicks two times, we first have to clear the previous
    # output:
    clear_output()
    display(container)
    
    ## Initialisation des variables fixées pour la suite :
    num_weeks = 1
    tagg_heure = ["0:00-1:00", "1:00-2:00", "2:00-3:00", "3:00-4:00","4:00-5:00","5:00-6:00","6:00-7:00","7:00-8:00","8:00-9:00","9:00-10:00","10:00-11:00","11:00-12:00","12:00-13:00","13:00-14:00","14:00-15:00","15:00-16:00","16:00-17:00","17:00-18:00","18:00-19:00","19:00-20:00","20:00-21:00","21:00-22:00","22:00-23:00","23:00-00:00"]
    zones = ["Avenue des Champs Elysées", "Sts-Pères", "Convention"]
    zones_heures={}
    for index, zone in enumerate(zones):
        zones_heures[zone] = [i for i in range(len(tagg_heure)*num_weeks*7)]
    
    
    ## Récupération des valeurs des widgets :
    nb_livreurs = slider_livreurs.value
    # On pioche aléatoirement parmi une liste de prénoms :
    livreurs = dict(zip(range(nb_livreurs), get_list_prenoms_courants(k=nb_livreurs)))
    
    Creneaux = ["R"] + tagg_heure
    
    
    # Récupération de l'intervalle sélectionné dans le widget associé :
    min_shift_per_day= shifts_slider.value[0]
    max_shift_per_day = shifts_slider.value[1]
    
    
    # Récupération du nombre de colis demandé par jour et par zone :
    list_nb_colis_par_jour_ACE = [dic_sliders_colis['Champs-Elysées'][i].value for i in range(7)]
    nb_colis_ACE = dict(zip(range(7), list_nb_colis_par_jour_ACE))
    
    list_nb_colis_par_jour_Sts = [dic_sliders_colis['Sts'][i].value for i in range(7)]
    nb_colis_Sts = dict(zip(range(7), list_nb_colis_par_jour_Sts))
    
    list_nb_colis_par_jour_convention = [dic_sliders_colis['Convention'][i].value for i in range(7)]
    nb_colis_convention = dict(zip(range(7), list_nb_colis_par_jour_convention))
    
    
    daily_cover_demands={}
    for d in range (num_weeks*7):
        for zone in zones:
            if zone == "Avenue des Champs Elysées":
                daily_cover_demands[d,zone]=nb_colis_ACE[d]
            if zone == "Sts-Pères":
                daily_cover_demands[d,zone]=nb_colis_Sts[d]
            if zone == "Convention":
                daily_cover_demands[d,zone]=nb_colis_convention[d]
                
    # paramètres temps de travails
    # contrainte_tp = [ [task1, task2,....], [shift1, shift2,...], total_nb_of_hours, nb_of_hours_min, nb_of_hours_max, min_penalty, max_penalty]
    contrainte_tp = []
    
    contrainte_tp.append([zone for zone in zones])
    contrainte_tp.append([shifts for shifts in range(len(tagg_heure))])
    
    contrainte_tp.append(35)
    contrainte_tp.append(0)
    contrainte_tp.append(60)
    contrainte_tp.append(15)
    contrainte_tp.append(15)

    shift_hours = {"0:00-1:00" : 1, "1:00-2:00":1, "2:00-3:00":1, "3:00-4:00":1,
                   "4:00-5:00":1,"5:00-6:00":1,"6:00-7:00":1,"7:00-8:00":1,
                   "8:00-9:00":1,"9:00-10:00":1,"10:00-11:00":1,"11:00-12:00":1,
                   "12:00-13:00":1,"13:00-14:00":1,"14:00-15:00":1,"15:00-16:00":1,
                   "16:00-17:00":1,"17:00-18:00":1,"18:00-19:00":1,"19:00-20:00":1,
                   "20:00-21:00":1,"21:00-22:00":1,"22:00-23:00":1,"23:00-00:00":1}

    df = pd.concat([pd.read_pickle('data/df_train.pkl'), pd.read_pickle('data/df_test.pkl')], axis=0)
    df_ACE = df[df['filename']=='champs-elysees.csv']
    df_Sts = df[df['filename']=='sts.csv']
    df_convention = df[df['filename']=='convention.csv']
    
    debut = pd.Timestamp('2020-11-23')
    fin = pd.Timestamp('2020-11-30')
    

    df_ACE = df_ACE[(df_ACE['Date et heure de comptage'] >= debut)]
    df_ACE = df_ACE[(df_ACE['Date et heure de comptage'] <= fin)]
    tx_occup_ACE = np.array(df_ACE["Taux d'occupation"])
    debit_ACE = np.array(df_ACE['Débit horaire'])

    df_Sts = df_Sts[(df_Sts['Date et heure de comptage'] >= debut)]
    df_Sts = df_Sts[(df_Sts['Date et heure de comptage'] <= fin)]
    tx_occup_Sts = np.array(df_Sts["Taux d'occupation"])
    debit_Sts = np.array(df_Sts['Débit horaire'])

    df_convention = df_convention[(df_convention['Date et heure de comptage'] >= debut)]
    df_convention = df_convention[(df_convention['Date et heure de comptage'] <= fin)]
    tx_occup_convention = np.array(df_convention["Taux d'occupation"])
    debit_convention = np.array(df_convention['Débit horaire'])

    occ_rues = {}
    for zone in zones:
        for d in range (num_weeks*7):
            for s in range(len(tagg_heure)):
                if zone == "Avenue des Champs Elysées":
                    occ_rues[s, d, zone] = int((tx_occup_ACE[d*len(tagg_heure)+s]/debit_ACE[d*len(tagg_heure)+s])*1000)
                if zone == "Sts-Pères":
                    occ_rues[s, d, zone] = int((tx_occup_Sts[d*len(tagg_heure)+s]/debit_Sts[d*len(tagg_heure)+s])*1000)
                if zone == "Convention":
                    occ_rues[s, d, zone] = int((tx_occup_convention[d * len(tagg_heure) + s] / debit_convention[d * len(tagg_heure) + s])*1000)

    waiting_time = 60
    
    occ_rues = {}
    for zone in zones:
        for d in range (num_weeks*7):
            for s in range(len(tagg_heure)):
                if zone == "Avenue des Champs Elysées":
                    occ_rues[s, d, zone] = int(tx_occup_ACE[d*len(tagg_heure)+s]*10)
                if zone == "Sts-Pères":
                    occ_rues[s, d, zone] = int(tx_occup_Sts[d*len(tagg_heure)+s]*10)
                if zone == "Convention":
                    occ_rues[s, d, zone] = int(tx_occup_convention[d * len(tagg_heure) + s]*10)
    
    
    
    ## Appel de la fonction de résolution du schedule
    solver, work, status = solve_shift_scheduling(livreurs, Creneaux, num_weeks, zones, 
                                                  max_shift_per_day, min_shift_per_day, 
                                                  contrainte_tp, daily_cover_demands,
                                                  occ_rues, waiting_time,shift_hours)

    
    ## Export du fichier Excel résultant
    all_shifts=num_weeks * len(tagg_heure) * 7
    wb = print_solution_blockwise_del(zones, all_shifts, livreurs, solver, work, tagg_heure, occ_rues)
    wb.save('planning.xls')
    
    
    ## Lecture et affichage du Excel obtenu :
    df = pd.read_excel('planning.xls', skiprows=3)
    
    df = df.fillna('')
    df.to_html('planning.html')
    
    display(HTML(filename="planning.html"))
    
    return

display(container)

Solution 0, time = 0.07 s, objective = 3360


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Jours,Heures,Avenue des Champs Elysées,Sts-Pères,Convention,
1,,0:00-1:00,,,,
2,,1:00-2:00,,,,
3,,2:00-3:00,,,,
4,,3:00-4:00,,,,
5,,4:00-5:00,,,,
6,,5:00-6:00,,,,
7,,6:00-7:00,,,,
8,,7:00-8:00,,,,
9,,8:00-9:00,,,,


# Visualisation du résultat

In [4]:
df = pd.read_excel('planning.xls', skiprows=4)

df = df.fillna('')




def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

styles = [
    
    hover(),
    dict(selector="th", props=[("font-size", "150%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom")]),
    #dict(selector = ".col0", props = [('display', 'none')]), 
    dict(selector = "th:first-child", props = [('display', 'none')])
]


html = (df.style.set_table_styles(styles)
          .set_caption("Tableau des plannings optmisés"))
html

,Jours,Heures,Avenue des Champs Elysées,Sts-Pères,Convention,Unnamed: 5
0,,0:00-1:00,Charlotte,,,
1,,1:00-2:00,Yvette,,Jeannine,
2,,2:00-3:00,Jeannine,,Yvette,
3,,3:00-4:00,Charlotte,,Gisèle,
4,,4:00-5:00,Yvette,,Anthony,
5,,5:00-6:00,Yvette,,Charlotte,
6,,6:00-7:00,,,,
7,,7:00-8:00,,,,
8,,8:00-9:00,,,,
9,,9:00-10:00,,,,
